In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

In [5]:
def segmentation(image):
    _, threshold = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    outer_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(outer_contour)
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image
    
def rotation(image, direction, s, e):
    #- kanan #+ kiri
    angle = random.randint(s, e)
    angle *= direction
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    return rotated_image

def crop_top(image, s,e):
    crop_percentage = random.randint(s,e)
    height = image.shape[0]
    crop_height = int(height * (crop_percentage/100))
    return image[crop_height:, :]

def crop_bottom(image, s,e):
    crop_percentage = random.randint(s,e)
    height = image.shape[0]
    crop_height = int(height * (1-crop_percentage/100))
    return image[:crop_height, :]

def crop_left(image, s,e):
    crop_percentage = random.randint(s,e)
    width = image.shape[1]
    crop_width = int(width * (crop_percentage/100))
    return image[:, crop_width:]

def crop_right(image, s,e):
    crop_percentage = random.randint(s,e)
    width = image.shape[1]
    crop_width = int(width * (1-crop_percentage/100))
    return image[:, :crop_width]

def perspective_top(image, s, e):
    percentage = random.randint(s,e)
    height, width = image.shape[:2]
    top_width = int(width * percentage/250)
    src_points = np.float32([[0, height], [width, height], [top_width, 0], [width - top_width, 0]])
    dst_points = np.float32([[0, height], [width, height], [0, 0], [width, 0]])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    result = cv2.warpPerspective(image, matrix, (width, height))
    return result

def perspective_bottom(image, s, e):
    percentage = random.randint(s, e)
    height, width = image.shape[:2]
    bottom_width = int(width * percentage / 250)
    src_points = np.float32([[0, 0], [width, 0], [bottom_width, height], [width - bottom_width, height]])
    dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    result = cv2.warpPerspective(image, matrix, (width, height))
    return result

def perspective_right(image, s, e):
    percentage = random.randint(s, e)
    height, width = image.shape[:2]
    right_height = int(height * percentage / 250)
    src_points = np.float32([[0, 0], [0, height], [width, right_height], [width, height - right_height]])
    dst_points = np.float32([[0, 0], [0, height], [width, 0], [width, height]])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    result = cv2.warpPerspective(image, matrix, (width, height))
    return result

def perspective_left(image, s, e):
    percentage = random.randint(s, e)
    height, width = image.shape[:2]
    left_height = int(height * percentage / 250)
    src_points = np.float32([[0, left_height], [0, height - left_height], [width, 0], [width, height]])
    dst_points = np.float32([[0, 0], [0, height], [width, left_height], [width, height - left_height]])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    result = cv2.warpPerspective(image, matrix, (width, height))
    return result

def salt_pepper_noise(image, s, e):
    probability = random.randint(s, e) / 100
    noise = np.random.choice([0, 255], size=image.shape, p=[probability, 1 - probability])
    noisy_image = cv2.add(image, noise, dtype=cv2.CV_8U)
    return noisy_image

def poisson_noise(image, s, e):
    lam = random.randint(s, e)
    noise = np.random.poisson(lam, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise, dtype=cv2.CV_8U)
    return noisy_image

def black_noise(image, s, e):
    probability = random.uniform(s, e)/100
    noise = np.random.choice([0, 255], size=image.shape[:2], p=[probability, 1 - probability])
    noisy_image = image.copy()
    noisy_image[np.where(noise == 0)] = 0
    return noisy_image

def gaussian_noise(image, ms, me, ss, se):
    mean = random.randint(ms, me)
    std_dev = random.randint(ss, se)
    noise = np.random.normal(mean, std_dev, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise, dtype=cv2.CV_8U)
    return noisy_image

def speckle_noise(image, ms, me, ss, se):
    mean = random.randint(ms, me)/100
    std_dev = random.randint(ss, se)/100
    h, w = image.shape[:2]
    noise = np.random.normal(mean, std_dev, (h, w)).astype(np.float32)
    noisy_image = image + image * noise
    return noisy_image.astype(np.uint8)

In [11]:
def augmentation(img):            
    imgs = []
    imgs.append(segmentation(img))
    
    imgs.append(rotation(segmentation(img), 1, 2, 6))
    imgs.append(rotation(segmentation(img), -1, 2, 6))
    imgs.append(segmentation(rotation(img, 1, 3, 7)))
    imgs.append(segmentation(rotation(img, -1, 3, 7)))
    
    imgs.append(crop_top(segmentation(img), 4,7))
    imgs.append(crop_bottom(segmentation(img), 4,7))
    imgs.append(crop_right(segmentation(img), 4,7))
    imgs.append(crop_left(segmentation(img), 4,7))

    imgs.append(crop_right(crop_top(segmentation(img), 4,7), 4,7))
    imgs.append(crop_left(crop_top(segmentation(img), 4,7), 4,7))
    imgs.append(crop_right(crop_bottom(segmentation(img), 4,7), 4,7))
    imgs.append(crop_left(crop_bottom(segmentation(img), 4,7), 4,7))

    imgs.append(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7))
    imgs.append(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7))
    imgs.append(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7))
    imgs.append(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7))

    imgs.append(perspective_top(segmentation(img), 40,50))
    imgs.append(perspective_bottom(segmentation(img), 40,50))
    imgs.append(perspective_right(segmentation(img), 20,30))
    imgs.append(perspective_left(segmentation(img), 20,30))

    imgs.append(segmentation(perspective_top(img, 40,50)))
    imgs.append(segmentation(perspective_bottom(img, 40,50)))
    imgs.append(segmentation(perspective_right(img, 20,30)))
    imgs.append(segmentation(perspective_left(img, 20,30)))
                                                        
    imgs.append(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)))
    imgs.append(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)))
    imgs.append(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)))
    imgs.append(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)))

    imgs.append(perspective_right(perspective_top(segmentation(img), 20,30), 10,20))
    imgs.append(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20))
    imgs.append(perspective_left(perspective_top(segmentation(img), 20,30), 10,20))
    imgs.append(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20))

    # ######## start salt_pepper_noise #############
    # imgs.append(salt_pepper_noise(segmentation(img), 75, 83))
    
    # imgs.append(salt_pepper_noise(rotation(segmentation(img), 1, 2, 6), 75, 83))
    # imgs.append(salt_pepper_noise(rotation(segmentation(img), -1, 2, 6), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(rotation(img, 1, 3, 7)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(rotation(img, -1, 3, 7)), 75, 83))
    
    # imgs.append(salt_pepper_noise(crop_top(segmentation(img), 4,7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_bottom(segmentation(img), 4,7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_right(segmentation(img), 4,7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_left(segmentation(img), 4,7), 75, 83))

    # imgs.append(salt_pepper_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7), 75, 83))

    # imgs.append(salt_pepper_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7), 75, 83))
    # imgs.append(salt_pepper_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7), 75, 83))

    # imgs.append(salt_pepper_noise(perspective_top(segmentation(img), 40,50), 75, 83))
    # imgs.append(salt_pepper_noise(perspective_bottom(segmentation(img), 40,50), 75, 83))
    # imgs.append(salt_pepper_noise(perspective_right(segmentation(img), 20,30), 75, 83))
    # imgs.append(salt_pepper_noise(perspective_left(segmentation(img), 20,30), 75, 83))

    # imgs.append(salt_pepper_noise(segmentation(perspective_top(img, 40,50)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(perspective_bottom(img, 40,50)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(perspective_right(img, 20,30)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(perspective_left(img, 20,30)), 75, 83))

    # imgs.append(salt_pepper_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)), 75, 83))
    # imgs.append(salt_pepper_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)), 75, 83))

    # imgs.append(salt_pepper_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20), 75, 83))
    # imgs.append(salt_pepper_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20), 75, 83))
    # imgs.append(salt_pepper_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20), 75, 83))
    # imgs.append(salt_pepper_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20), 75, 83))

    # ######## start poisson_noise #############
    imgs.append(poisson_noise(segmentation(img), 90, 110))
    
    imgs.append(poisson_noise(rotation(segmentation(img), 1, 2, 6), 90, 110))
    imgs.append(poisson_noise(rotation(segmentation(img), -1, 2, 6), 90, 110))
    imgs.append(poisson_noise(segmentation(rotation(img, 1, 3, 7)), 90, 110))
    imgs.append(poisson_noise(segmentation(rotation(img, -1, 3, 7)), 90, 110))
    
    imgs.append(poisson_noise(crop_top(segmentation(img), 4,7), 90, 110))
    imgs.append(poisson_noise(crop_bottom(segmentation(img), 4,7), 90, 110))
    imgs.append(poisson_noise(crop_right(segmentation(img), 4,7), 90, 110))
    imgs.append(poisson_noise(crop_left(segmentation(img), 4,7), 90, 110))

    imgs.append(poisson_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7), 90, 110))
    imgs.append(poisson_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7), 90, 110))
    imgs.append(poisson_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7), 90, 110))
    imgs.append(poisson_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7), 90, 110))

    imgs.append(poisson_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7), 90, 110))
    imgs.append(poisson_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7), 90, 110))
    imgs.append(poisson_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7), 90, 110))
    imgs.append(poisson_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7), 90, 110))

    imgs.append(poisson_noise(perspective_top(segmentation(img), 40,50), 90, 110))
    imgs.append(poisson_noise(perspective_bottom(segmentation(img), 40,50), 90, 110))
    imgs.append(poisson_noise(perspective_right(segmentation(img), 20,30), 90, 110))
    imgs.append(poisson_noise(perspective_left(segmentation(img), 20,30), 90, 110))

    imgs.append(poisson_noise(segmentation(perspective_top(img, 40,50)), 90, 110))
    imgs.append(poisson_noise(segmentation(perspective_bottom(img, 40,50)), 90, 110))
    imgs.append(poisson_noise(segmentation(perspective_right(img, 20,30)), 90, 110))
    imgs.append(poisson_noise(segmentation(perspective_left(img, 20,30)), 90, 110))

    imgs.append(poisson_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)), 90, 110))
    imgs.append(poisson_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)), 90, 110))
    imgs.append(poisson_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)), 90, 110))
    imgs.append(poisson_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)), 90, 110))

    imgs.append(poisson_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20), 90, 110))
    imgs.append(poisson_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20), 90, 110))
    imgs.append(poisson_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20), 90, 110))
    imgs.append(poisson_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20), 90, 110))

    ###### start black_noise #############
    imgs.append(black_noise(segmentation(img),20, 40))
    
    imgs.append(black_noise(rotation(segmentation(img), 1, 2, 6),20, 40))
    imgs.append(black_noise(rotation(segmentation(img), -1, 2, 6),20, 40))
    imgs.append(black_noise(segmentation(rotation(img, 1, 3, 7)),20, 40))
    imgs.append(black_noise(segmentation(rotation(img, -1, 3, 7)),20, 40))
    
    imgs.append(black_noise(crop_top(segmentation(img), 4,7),20, 40))
    imgs.append(black_noise(crop_bottom(segmentation(img), 4,7),20, 40))
    imgs.append(black_noise(crop_right(segmentation(img), 4,7),20, 40))
    imgs.append(black_noise(crop_left(segmentation(img), 4,7),20, 40))

    imgs.append(black_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7),20, 40))
    imgs.append(black_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7),20, 40))
    imgs.append(black_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7),20, 40))
    imgs.append(black_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7),20, 40))

    imgs.append(black_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),20, 40))
    imgs.append(black_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),20, 40))
    imgs.append(black_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7),20, 40))
    imgs.append(black_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7),20, 40))

    imgs.append(black_noise(perspective_top(segmentation(img), 40,50),20, 40))
    imgs.append(black_noise(perspective_bottom(segmentation(img), 40,50),20, 40))
    imgs.append(black_noise(perspective_right(segmentation(img), 20,30),20, 40))
    imgs.append(black_noise(perspective_left(segmentation(img), 20,30),20, 40))

    imgs.append(black_noise(segmentation(perspective_top(img, 40,50)),20, 40))
    imgs.append(black_noise(segmentation(perspective_bottom(img, 40,50)),20, 40))
    imgs.append(black_noise(segmentation(perspective_right(img, 20,30)),20, 40))
    imgs.append(black_noise(segmentation(perspective_left(img, 20,30)),20, 40))

    imgs.append(black_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)),20, 40))
    imgs.append(black_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)),20, 40))
    imgs.append(black_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)),20, 40))
    imgs.append(black_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)),20, 40))

    imgs.append(black_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20),20, 40))
    imgs.append(black_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20),20, 40))
    imgs.append(black_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20),20, 40))
    imgs.append(black_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20),20, 40))

    ######## start gaussian_noise #############
    imgs.append(gaussian_noise(segmentation(img),12,20,12,20))
    
    imgs.append(gaussian_noise(rotation(segmentation(img), 1, 2, 6),12,20,12,20))
    imgs.append(gaussian_noise(rotation(segmentation(img), -1, 2, 6),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(rotation(img, 1, 3, 7)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(rotation(img, -1, 3, 7)),12,20,12,20))
    
    imgs.append(gaussian_noise(crop_top(segmentation(img), 4,7),12,20,12,20))
    imgs.append(gaussian_noise(crop_bottom(segmentation(img), 4,7),12,20,12,20))
    imgs.append(gaussian_noise(crop_right(segmentation(img), 4,7),12,20,12,20))
    imgs.append(gaussian_noise(crop_left(segmentation(img), 4,7),12,20,12,20))

    imgs.append(gaussian_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7),12,20,12,20))
    imgs.append(gaussian_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7),12,20,12,20))
    imgs.append(gaussian_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7),12,20,12,20))
    imgs.append(gaussian_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7),12,20,12,20))

    imgs.append(gaussian_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),12,20,12,20))
    imgs.append(gaussian_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),12,20,12,20))
    imgs.append(gaussian_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7),12,20,12,20))
    imgs.append(gaussian_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7),12,20,12,20))

    imgs.append(gaussian_noise(perspective_top(segmentation(img), 40,50),12,20,12,20))
    imgs.append(gaussian_noise(perspective_bottom(segmentation(img), 40,50),12,20,12,20))
    imgs.append(gaussian_noise(perspective_right(segmentation(img), 20,30),12,20,12,20))
    imgs.append(gaussian_noise(perspective_left(segmentation(img), 20,30),12,20,12,20))

    imgs.append(gaussian_noise(segmentation(perspective_top(img, 40,50)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(perspective_bottom(img, 40,50)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(perspective_right(img, 20,30)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(perspective_left(img, 20,30)),12,20,12,20))

    imgs.append(gaussian_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)),12,20,12,20))
    imgs.append(gaussian_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)),12,20,12,20))

    imgs.append(gaussian_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20),12,20,12,20))
    imgs.append(gaussian_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20),12,20,12,20))
    imgs.append(gaussian_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20),12,20,12,20))
    imgs.append(gaussian_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20),12,20,12,20))

    ####### start speckle_noise #############
    imgs.append(speckle_noise(segmentation(img),5,5,5,15))
    
    imgs.append(speckle_noise(rotation(segmentation(img), 1, 2, 6),5,5,5,15))
    imgs.append(speckle_noise(rotation(segmentation(img), -1, 2, 6),5,5,5,15))
    imgs.append(speckle_noise(segmentation(rotation(img, 1, 3, 7)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(rotation(img, -1, 3, 7)),5,5,5,15))
    
    imgs.append(speckle_noise(crop_top(segmentation(img), 4,7),5,5,5,15))
    imgs.append(speckle_noise(crop_bottom(segmentation(img), 4,7),5,5,5,15))
    imgs.append(speckle_noise(crop_right(segmentation(img), 4,7),5,5,5,15))
    imgs.append(speckle_noise(crop_left(segmentation(img), 4,7),5,5,5,15))

    imgs.append(speckle_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7),5,5,5,15))
    imgs.append(speckle_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7),5,5,5,15))
    imgs.append(speckle_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7),5,5,5,15))
    imgs.append(speckle_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7),5,5,5,15))

    imgs.append(speckle_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),5,5,5,15))
    imgs.append(speckle_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),5,5,5,15))
    imgs.append(speckle_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7),5,5,5,15))
    imgs.append(speckle_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7),5,5,5,15))

    imgs.append(speckle_noise(perspective_top(segmentation(img), 40,50),5,5,5,15))
    imgs.append(speckle_noise(perspective_bottom(segmentation(img), 40,50),5,5,5,15))
    imgs.append(speckle_noise(perspective_right(segmentation(img), 20,30),5,5,5,15))
    imgs.append(speckle_noise(perspective_left(segmentation(img), 20,30),5,5,5,15))

    imgs.append(speckle_noise(segmentation(perspective_top(img, 40,50)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(perspective_bottom(img, 40,50)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(perspective_right(img, 20,30)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(perspective_left(img, 20,30)),5,5,5,15))

    imgs.append(speckle_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)),5,5,5,15))
    imgs.append(speckle_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)),5,5,5,15))

    imgs.append(speckle_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20),5,5,5,15))
    imgs.append(speckle_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20),5,5,5,15))
    imgs.append(speckle_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20),5,5,5,15))
    imgs.append(speckle_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20),5,5,5,15))

    # ####### start mix salt_pepper and blacknoise #############
    imgs.append(black_noise(salt_pepper_noise(segmentation(img),70,80), 10,20))
    
    imgs.append(black_noise(salt_pepper_noise(rotation(segmentation(img), 1, 2, 6),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(rotation(segmentation(img), -1, 2, 6),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(rotation(img, 1, 3, 7)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(rotation(img, -1, 3, 7)),70,80), 10,20))
    
    imgs.append(black_noise(salt_pepper_noise(crop_top(segmentation(img), 4,7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_bottom(segmentation(img), 4,7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_right(segmentation(img), 4,7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_left(segmentation(img), 4,7),70,80), 10,20))

    imgs.append(black_noise(salt_pepper_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7),70,80), 10,20))

    imgs.append(black_noise(salt_pepper_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7),70,80), 10,20))

    imgs.append(black_noise(salt_pepper_noise(perspective_top(segmentation(img), 40,50),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(perspective_bottom(segmentation(img), 40,50),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(perspective_right(segmentation(img), 20,30),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(perspective_left(segmentation(img), 20,30),70,80), 10,20))

    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_top(img, 40,50)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_bottom(img, 40,50)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_right(img, 20,30)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_left(img, 20,30)),70,80), 10,20))

    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)),70,80), 10,20))

    imgs.append(black_noise(salt_pepper_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20),70,80), 10,20))
    imgs.append(black_noise(salt_pepper_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20),70,80), 10,20))

    ####### start mix poisson_noise and blacknoise #############
  
    imgs.append(black_noise(poisson_noise(segmentation(img),30,70), 20,60))
    
    imgs.append(black_noise(poisson_noise(rotation(segmentation(img), 1, 2, 6),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(rotation(segmentation(img), -1, 2, 6),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(rotation(img, 1, 3, 7)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(rotation(img, -1, 3, 7)),30,70), 20,60))
    
    imgs.append(black_noise(poisson_noise(crop_top(segmentation(img), 4,7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_bottom(segmentation(img), 4,7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_right(segmentation(img), 4,7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_left(segmentation(img), 4,7),30,70), 20,60))

    imgs.append(black_noise(poisson_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7),30,70), 20,60))

    imgs.append(black_noise(poisson_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7),30,70), 20,60))

    imgs.append(black_noise(poisson_noise(perspective_top(segmentation(img), 40,50),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(perspective_bottom(segmentation(img), 40,50),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(perspective_right(segmentation(img), 20,30),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(perspective_left(segmentation(img), 20,30),30,70), 20,60))

    imgs.append(black_noise(poisson_noise(segmentation(perspective_top(img, 40,50)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_bottom(img, 40,50)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_right(img, 20,30)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_left(img, 20,30)),30,70), 20,60))

    imgs.append(black_noise(poisson_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)),30,70), 20,60))

    imgs.append(black_noise(poisson_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20),30,70), 20,60))
    imgs.append(black_noise(poisson_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20),30,70), 20,60))

    ###### start black_noise and gaussian_noise #############
    
    imgs.append(black_noise(gaussian_noise(segmentation(img),10,30, 10,40), 10,70))
    
    imgs.append(black_noise(gaussian_noise(rotation(segmentation(img), 1, 2, 6),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(rotation(segmentation(img), -1, 2, 6),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(rotation(img, 1, 3, 7)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(rotation(img, -1, 3, 7)),10,30, 10,40), 10,70))
    
    imgs.append(black_noise(gaussian_noise(crop_top(segmentation(img), 4,7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_bottom(segmentation(img), 4,7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_right(segmentation(img), 4,7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_left(segmentation(img), 4,7),10,30, 10,40), 10,70))

    imgs.append(black_noise(gaussian_noise(crop_right(crop_top(segmentation(img), 4,7), 4,7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_left(crop_top(segmentation(img), 4,7), 4,7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_right(crop_bottom(segmentation(img), 4,7), 4,7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_left(crop_bottom(segmentation(img), 4,7), 4,7),10,30, 10,40), 10,70))

    imgs.append(black_noise(gaussian_noise(crop_top(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_bottom(crop_right(crop_left(segmentation(img), 4,7), 4,7), 4, 7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_top(crop_bottom(crop_left(segmentation(img), 4,7), 4,7), 4, 7),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(crop_top(crop_bottom(crop_right(segmentation(img), 4,7), 4,7), 4, 7),10,30, 10,40), 10,70))

    imgs.append(black_noise(gaussian_noise(perspective_top(segmentation(img), 40,50),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(perspective_bottom(segmentation(img), 40,50),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(perspective_right(segmentation(img), 20,30),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(perspective_left(segmentation(img), 20,30),10,30, 10,40), 10,70))

    imgs.append(black_noise(gaussian_noise(segmentation(perspective_top(img, 40,50)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_bottom(img, 40,50)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_right(img, 20,30)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_left(img, 20,30)),10,30, 10,40), 10,70))

    imgs.append(black_noise(gaussian_noise(segmentation(perspective_right(perspective_top(img, 40,50), 20,30)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_right(perspective_bottom(img, 40,50), 20,30)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_left(perspective_top(img, 40,50), 20,30)),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_left(perspective_bottom(img, 40,50), 20,30)),10,30, 10,40), 10,70))

    imgs.append(black_noise(gaussian_noise(perspective_right(perspective_top(segmentation(img), 20,30), 10,20),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(perspective_right(perspective_bottom(segmentation(img), 20,30), 10,20),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(perspective_left(perspective_top(segmentation(img), 20,30), 10,20),10,30, 10,40), 10,70))
    imgs.append(black_noise(gaussian_noise(perspective_left(perspective_bottom(segmentation(img), 20,30), 10,20),10,30, 10,40), 10,70))

    return imgs

In [4]:
def validation(img):            
    imgs = []

    # ####### start mix salt_pepper and blacknoise #############
    imgs.append(black_noise(salt_pepper_noise(segmentation(img),40,60), 30,40))
    
    imgs.append(black_noise(salt_pepper_noise(rotation(segmentation(img), 1, 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(rotation(segmentation(img), -1, 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(rotation(img, 1, 8, 9)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(rotation(img, -1, 8, 9)),40,60), 30,40))
    
    imgs.append(black_noise(salt_pepper_noise(crop_top(segmentation(img), 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_bottom(segmentation(img), 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_right(segmentation(img), 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_left(segmentation(img), 8, 9),40,60), 30,40))

    imgs.append(black_noise(salt_pepper_noise(crop_right(crop_top(segmentation(img), 8, 9), 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_left(crop_top(segmentation(img), 8, 9), 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_right(crop_bottom(segmentation(img), 8, 9), 8, 9),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_left(crop_bottom(segmentation(img), 8, 9), 8, 9),40,60), 30,40))

    imgs.append(black_noise(salt_pepper_noise(crop_top(crop_right(crop_left(segmentation(img), 8, 9), 8, 9), 4, 7),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_bottom(crop_right(crop_left(segmentation(img), 8, 9), 8, 9), 4, 7),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_top(crop_bottom(crop_left(segmentation(img), 8, 9), 8, 9), 4, 7),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(crop_top(crop_bottom(crop_right(segmentation(img), 8, 9), 8, 9), 4, 7),40,60), 30,40))

    imgs.append(black_noise(salt_pepper_noise(perspective_top(segmentation(img), 60,80),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(perspective_bottom(segmentation(img), 60,80),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(perspective_right(segmentation(img), 40,50),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(perspective_left(segmentation(img), 40,50),40,60), 30,40))

    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_top(img, 60,80)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_bottom(img, 60,80)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_right(img, 40,50)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_left(img, 40,50)),40,60), 30,40))

    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_right(perspective_top(img, 60,80), 40,50)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_right(perspective_bottom(img, 60,80), 40,50)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_left(perspective_top(img, 60,80), 40,50)),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(segmentation(perspective_left(perspective_bottom(img, 60,80), 40,50)),40,60), 30,40))

    imgs.append(black_noise(salt_pepper_noise(perspective_right(perspective_top(segmentation(img), 40,50), 30,40),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(perspective_right(perspective_bottom(segmentation(img), 40,50), 30,40),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(perspective_left(perspective_top(segmentation(img), 40,50), 30,40),40,60), 30,40))
    imgs.append(black_noise(salt_pepper_noise(perspective_left(perspective_bottom(segmentation(img), 40,50), 30,40),40,60), 30,40))

    ####### start mix poisson_noise and blacknoise #############
    imgs.append(black_noise(poisson_noise(segmentation(img),80,90), 70,90))
    
    imgs.append(black_noise(poisson_noise(rotation(segmentation(img), 1, 7, 9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(rotation(segmentation(img), -1, 7, 9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(rotation(img, 1, 8, 9)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(rotation(img, -1, 8, 9)),80,90), 70,90))
    
    imgs.append(black_noise(poisson_noise(crop_top(segmentation(img), 8,9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_bottom(segmentation(img), 8,9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_right(segmentation(img), 8,9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_left(segmentation(img), 8,9),80,90), 70,90))

    imgs.append(black_noise(poisson_noise(crop_right(crop_top(segmentation(img), 8,9), 8,9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_left(crop_top(segmentation(img), 8,9), 8,9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_right(crop_bottom(segmentation(img), 8,9), 8,9),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_left(crop_bottom(segmentation(img), 8,9), 8,9),80,90), 70,90))

    imgs.append(black_noise(poisson_noise(crop_top(crop_right(crop_left(segmentation(img), 8,9), 8,9), 4, 7),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_bottom(crop_right(crop_left(segmentation(img), 8,9), 8,9), 4, 7),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_top(crop_bottom(crop_left(segmentation(img), 8,9), 8,9), 4, 7),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(crop_top(crop_bottom(crop_right(segmentation(img), 8,9), 8,9), 4, 7),80,90), 70,90))

    imgs.append(black_noise(poisson_noise(perspective_top(segmentation(img), 60,70),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(perspective_bottom(segmentation(img), 60,70),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(perspective_right(segmentation(img), 40,50),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(perspective_left(segmentation(img), 40,50),80,90), 70,90))

    imgs.append(black_noise(poisson_noise(segmentation(perspective_top(img, 60,70)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_bottom(img, 60,70)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_right(img, 40,50)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_left(img, 40,50)),80,90), 70,90))

    imgs.append(black_noise(poisson_noise(segmentation(perspective_right(perspective_top(img, 60,70), 40,50)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_right(perspective_bottom(img, 60,70), 40,50)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_left(perspective_top(img, 60,70), 40,50)),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(segmentation(perspective_left(perspective_bottom(img, 60,70), 40,50)),80,90), 70,90))

    imgs.append(black_noise(poisson_noise(perspective_right(perspective_top(segmentation(img), 40,50), 10,20),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(perspective_right(perspective_bottom(segmentation(img), 40,50), 10,20),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(perspective_left(perspective_top(segmentation(img), 40,50), 10,20),80,90), 70,90))
    imgs.append(black_noise(poisson_noise(perspective_left(perspective_bottom(segmentation(img), 40,50), 10,20),80,90), 70,90))

    return imgs

In [5]:
def test(img):            
    imgs = []
    imgs.append(black_noise(gaussian_noise(segmentation(img),10,60, 10,90), 5,90))
    
    imgs.append(black_noise(gaussian_noise(rotation(segmentation(img), 1, 2, 9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(rotation(segmentation(img), -1, 2, 9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(rotation(img, 1, 2, 9)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(rotation(img, -1, 2, 9)),10,60, 10,90), 5,90))
    
    imgs.append(black_noise(gaussian_noise(crop_top(segmentation(img), 2,9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_bottom(segmentation(img), 2,9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_right(segmentation(img), 2,9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_left(segmentation(img), 2,9),10,60, 10,90), 5,90))

    imgs.append(black_noise(gaussian_noise(crop_right(crop_top(segmentation(img), 2,9), 2,9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_left(crop_top(segmentation(img), 2,9), 2,9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_right(crop_bottom(segmentation(img), 2,9), 2,9),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_left(crop_bottom(segmentation(img), 2,9), 2,9),10,60, 10,90), 5,90))

    imgs.append(black_noise(gaussian_noise(crop_top(crop_right(crop_left(segmentation(img), 2,9), 2,9), 4, 7),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_bottom(crop_right(crop_left(segmentation(img), 2,9), 2,9), 4, 7),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_top(crop_bottom(crop_left(segmentation(img), 2,9), 2,9), 4, 7),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(crop_top(crop_bottom(crop_right(segmentation(img), 2,9), 2,9), 4, 7),10,60, 10,90), 5,90))

    imgs.append(black_noise(gaussian_noise(perspective_top(segmentation(img), 20,70),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(perspective_bottom(segmentation(img), 20,70),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(perspective_right(segmentation(img), 10,50),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(perspective_left(segmentation(img), 10,50),10,60, 10,90), 5,90))

    imgs.append(black_noise(gaussian_noise(segmentation(perspective_top(img, 20,70)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_bottom(img, 20,70)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_right(img, 10,50)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_left(img, 10,50)),10,60, 10,90), 5,90))

    imgs.append(black_noise(gaussian_noise(segmentation(perspective_right(perspective_top(img, 20,70), 10,50)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_right(perspective_bottom(img, 20,70), 10,50)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_left(perspective_top(img, 20,70), 10,50)),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(segmentation(perspective_left(perspective_bottom(img, 20,70), 10,50)),10,60, 10,90), 5,90))

    imgs.append(black_noise(gaussian_noise(perspective_right(perspective_top(segmentation(img), 10,50), 10,20),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(perspective_right(perspective_bottom(segmentation(img), 10,50), 10,20),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(perspective_left(perspective_top(segmentation(img), 10,50), 10,20),10,60, 10,90), 5,90))
    imgs.append(black_noise(gaussian_noise(perspective_left(perspective_bottom(segmentation(img), 10,50), 10,20),10,60, 10,90), 5,90))

    imgs.append(speckle_noise(segmentation(img),6,9,2,25))
    
    imgs.append(speckle_noise(rotation(segmentation(img), 1, 1, 9),6,9,2,25))
    imgs.append(speckle_noise(rotation(segmentation(img), -1, 1, 9),6,9,2,25))
    imgs.append(speckle_noise(segmentation(rotation(img, 1, 1, 9)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(rotation(img, -1, 1, 9)),6,9,2,25))
    
    imgs.append(speckle_noise(crop_top(segmentation(img), 1,9),6,9,2,25))
    imgs.append(speckle_noise(crop_bottom(segmentation(img), 1,9),6,9,2,25))
    imgs.append(speckle_noise(crop_right(segmentation(img), 1,9),6,9,2,25))
    imgs.append(speckle_noise(crop_left(segmentation(img), 1,9),6,9,2,25))

    imgs.append(speckle_noise(crop_right(crop_top(segmentation(img), 1,9), 1,9),6,9,2,25))
    imgs.append(speckle_noise(crop_left(crop_top(segmentation(img), 1,9), 1,9),6,9,2,25))
    imgs.append(speckle_noise(crop_right(crop_bottom(segmentation(img), 1,9), 1,9),6,9,2,25))
    imgs.append(speckle_noise(crop_left(crop_bottom(segmentation(img), 1,9), 1,9),6,9,2,25))

    imgs.append(speckle_noise(crop_top(crop_right(crop_left(segmentation(img), 1,9), 1,9), 4, 7),6,9,2,25))
    imgs.append(speckle_noise(crop_bottom(crop_right(crop_left(segmentation(img), 1,9), 1,9), 4, 7),6,9,2,25))
    imgs.append(speckle_noise(crop_top(crop_bottom(crop_left(segmentation(img), 1,9), 1,9), 4, 7),6,9,2,25))
    imgs.append(speckle_noise(crop_top(crop_bottom(crop_right(segmentation(img), 1,9), 1,9), 4, 7),6,9,2,25))

    imgs.append(speckle_noise(perspective_top(segmentation(img), 10,90),6,9,2,25))
    imgs.append(speckle_noise(perspective_bottom(segmentation(img), 10,90),6,9,2,25))
    imgs.append(speckle_noise(perspective_right(segmentation(img), 10,90),6,9,2,25))
    imgs.append(speckle_noise(perspective_left(segmentation(img), 10,90),6,9,2,25))

    imgs.append(speckle_noise(segmentation(perspective_top(img, 10,90)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(perspective_bottom(img, 10,90)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(perspective_right(img, 10,90)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(perspective_left(img, 10,90)),6,9,2,25))

    imgs.append(speckle_noise(segmentation(perspective_right(perspective_top(img, 10,90), 10,90)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(perspective_right(perspective_bottom(img, 10,90), 10,90)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(perspective_left(perspective_top(img, 10,90), 10,90)),6,9,2,25))
    imgs.append(speckle_noise(segmentation(perspective_left(perspective_bottom(img, 10,90), 10,90)),6,9,2,25))

    imgs.append(speckle_noise(perspective_right(perspective_top(segmentation(img), 10,90), 10,20),6,9,2,25))
    imgs.append(speckle_noise(perspective_right(perspective_bottom(segmentation(img), 10,90), 10,20),6,9,2,25))
    imgs.append(speckle_noise(perspective_left(perspective_top(segmentation(img), 10,90), 10,20),6,9,2,25))
    imgs.append(speckle_noise(perspective_left(perspective_bottom(segmentation(img), 10,90), 10,20),6,9,2,25))

    return imgs

In [12]:
root_directory = r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\raw'
train_path = r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train'
valid_path = r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\valid'
test_path = r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\test'

def save_images(images, output_folder):
    # Membuat folder output jika belum ada
    os.makedirs(output_folder, exist_ok=True)

    # Menyimpan gambar-gambar ke dalam folder
    for i, img in enumerate(images):
        img_path = os.path.join(output_folder, f'{i+1}.jpg')
        cv2.imwrite(img_path, img)
        print(f'Simpan gambar {img_path}')

# Mengambil daftar folder di direktori utama
folders = os.listdir(root_directory)

# # Iterasi melalui setiap folder
for folder in folders:

    # Membuat path lengkap ke folder
    folder_path = os.path.join(root_directory, folder)

    print(folder_path)
    # Memeriksa apakah path adalah direktori
    if os.path.isdir(folder_path):
        # Mengambil path gambar di dalam folder
        image_path = os.path.join(folder_path, '0.png')
        
        # Membaca gambar menggunakan OpenCV
        image = cv2.imread(image_path)

        # Konversi grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Konversi Numpy
        image = np.array(image)
        
        # Melakukan augmentasi gambar
        train_images = augmentation(image)
        valid_images = validation(image)
        test_images = test(image)
        
        # Menentukan output folder sesuai dengan folder saat ini
        train_folder = os.path.join(train_path + '/' +  folder)
        valid_folder = os.path.join(valid_path + '/' + folder)
        test_folder = os.path.join(test_path + '/' +  folder)
        
        # Menyimpan gambar-gambar yang dihasilkan
        save_images(train_images, train_folder)
        save_images(valid_images, valid_folder)
        save_images(test_images, test_folder)

C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\raw\0
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\1.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\2.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\3.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\4.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\5.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\6.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\7.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\8.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\9.jpg
Simpan gambar C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train/0\10.jpg
Simpan gambar C:\Use